In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
import tensorflow as tf
matplotlib.rcParams["image.interpolation"] = None
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tifffile import imread
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible

from stardist import random_label_cmap, _draw_polygons, export_imagej_rois
from stardist.models import StarDist2D

np.random.seed(6)
lbl_cmap = random_label_cmap()

# Data

Based on training from step 2 (model already created), now we create prediction for each independant slice of the 3D confocal images to be processed and save the label. We will create 3D labels from this using the wand tool for example and giving one label to each 3D nucleus. 

In [10]:
X = sorted(glob('creating2Dlabelsfor3D/zone3/image/*.tif'))
X = list(map(imread,X))
print(len(X))

n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
axis_norm = (0,1)   # normalize channels independently
# axis_norm = (0,1,2) # normalize channels jointly
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))

31


In [11]:
# show all test images
if False:
    fig, ax = plt.subplots(7,8, figsize=(16,16))
    for i,(a,x) in enumerate(zip(ax.flat, X)):
        a.imshow(x if x.ndim==2 else x[...,0], cmap='gray')
        a.set_title(i)
    [a.axis('off') for a in ax.flat]
    plt.tight_layout()
None;

# Load trained model

If you trained your own StarDist model (and optimized its thresholds) via notebook [2_training.ipynb](2_training.ipynb), then please set `demo_model = False` below.

In [12]:

    model = StarDist2D(None, name='stardistGut', basedir='models')


Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.765736, nms_thresh=0.3.


## Prediction

Make sure to normalize the input image beforehand or supply a `normalizer` to the prediction function.

Calling `model.predict_instances` will
- predict object probabilities and star-convex polygon distances (see `model.predict` if you want those)
- perform non-maximum suppression (with overlap threshold `nms_thresh`) for polygons above object probability threshold `prob_thresh`.
- render all remaining polygon instances in a label image
- return the label instances image and also the details (coordinates, etc.) of all remaining polygons

In [13]:
for i in range (len(X)):
    img = normalize(X[i], 1,99.8, axis=axis_norm)
    labels, details = model.predict_instances(img)
    #plt.figure(figsize=(8,8)) 
    #plt.imshow(img if img.ndim==2 else img[...,0], clim=(0,1), cmap='gray')
    #plt.imshow(labels, cmap=lbl_cmap, alpha=0.5)
    #plt.axis('off');
    save_tiff_imagej_compatible('creating2Dlabelsfor3D\zone3\label\example_image'+str(i)+'.tif', img, axes='YX')
    save_tiff_imagej_compatible('creating2Dlabelsfor3D\zone3\label\example_labels'+str(i)+'.tif', labels, axes='YX')